# TrueLayer Data Engineer Challenge

### Reading in movies metadata, calculating budget/revenue ratio and extracting required fields

In [1]:
import pandas as pd
import requests
import subprocess

In [2]:
df = pd.read_csv('movies_metadata.csv')

# Converting budget column to numeric (float) from string and dropping all NaNs and 0.0 values. Prevents
# infinity in ratio calculation
df['budget'] = df['budget'].apply(pd.to_numeric, errors='coerce').dropna()
df = df[(df['budget'] != 0.0)]
df['ratio'] = df['revenue']/df['budget'].dropna()

# Dropping not required columns and selecting top 1000 ratio rows
cols_to_keep = ['title', 'budget', 'revenue', 'production_companies', 'vote_average', 'release_date', 'ratio']
cols_to_drop = [col for col in df.columns if col not in cols_to_keep]
df = df.drop(cols_to_drop, axis=1)

df = df.sort_values('ratio',ascending = False).head(1000).reset_index(drop=True)

/Users/janzajac/.local/share/virtualenvs/TrueLayer-ifjXq_FT/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,budget,production_companies,release_date,revenue,title,vote_average,ratio
0,1.0,[{'name': 'Twentieth Century Fox Film Corporat...,1987-11-06,12396383.0,Less Than Zero,6.1,1.239638e+07
1,1.0,"[{'name': 'United Artists', 'id': 60}, {'name'...",1936-02-05,8500000.0,Modern Times,8.1,8.500000e+06
2,8.0,"[{'name': 'Film It Suda', 'id': 5659}]",2005-08-04,33579813.0,Welcome to Dongmakgol,7.7,4.197477e+06
3,1.0,[],2012-03-30,2755584.0,Aquí Entre Nos,6.0,2.755584e+06
4,113.0,"[{'name': 'Columbia Pictures Corporation', 'id...",1986-06-18,115103979.0,"The Karate Kid, Part II",5.9,1.018619e+06


### Extracting title, url and abstract from wiki xml dump

In [4]:
# taken from https://towardsdatascience.com/wikipedia-data-science-working-with-the-worlds-largest-encyclopedia-c08efbac5f5c
import xml.sax


class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._content = {} # Dictionary where the movie name is key and url and abstract are values

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'url', 'abstract'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'doc':
            self._content[self._values['title'].strip('Wikipedia: ')] = {'url': self._values['url'], 'abstract': self._values['abstract']}


In [5]:
# Parsing xml line by line and extracting required tags rather than reading file into memory

data_path = 'enwiki-latest-abstract.xml'
# Object for handling xml
handler = WikiXmlHandler()
# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)
# Iteratively process file
for line in subprocess.Popen(['cat'], 
                              stdin = open(data_path), 
                              stdout = subprocess.PIPE).stdout:
    parser.feed(line)

In [6]:
# Looping through titles in movies metadata dataframe and extracting url and abstract

wiki_titles_found = []
for title in df['title']:
    try:
        wiki_dict = {'title': title, 'wiki_url':handler._content[title]['url'], 'wiki_abstract':handler._content[title]['abstract']} 
        wiki_titles_found.append(wiki_dict)
    except KeyError:
        pass

In [7]:
len(wiki_titles_found)

598

In [8]:
# Constructing df from wiki data, ready for join with initial dataframe

wiki_df = pd.DataFrame(wiki_titles_found)
wiki_df.head()

,title,wiki_url,wiki_abstract
0,Less Than Zero,https://en.wikipedia.org/wiki/Less_Than_Zero,Less Than Zero may refer to:
1,Modern Times,https://en.wikipedia.org/wiki/Modern_Times,Modern Times may refer to modern history.
2,Paranormal Activity,https://en.wikipedia.org/wiki/Paranormal_Activity,| writer = Oren Peli
3,Tarnation,https://en.wikipedia.org/wiki/Tarnation,Tarnation may refer to:
4,The Blair Witch Project,https://en.wikipedia.org/wiki/The_Blair_Witch_...,| producers =


In [9]:
# Joining the two dataframes together, ready for insert into Postgres database table
full_df = pd.merge(left=df, right=wiki_df, how='left', left_on='title', right_on='title')
full_df.head()

,budget,production_companies,release_date,revenue,title,vote_average,ratio,wiki_url,wiki_abstract
0,1.0,[{'name': 'Twentieth Century Fox Film Corporat...,1987-11-06,12396383.0,Less Than Zero,6.1,1.239638e+07,https://en.wikipedia.org/wiki/Less_Than_Zero,Less Than Zero may refer to:
1,1.0,"[{'name': 'United Artists', 'id': 60}, {'name'...",1936-02-05,8500000.0,Modern Times,8.1,8.500000e+06,https://en.wikipedia.org/wiki/Modern_Times,Modern Times may refer to modern history.
2,8.0,"[{'name': 'Film It Suda', 'id': 5659}]",2005-08-04,33579813.0,Welcome to Dongmakgol,7.7,4.197477e+06,NaN,NaN
3,1.0,[],2012-03-30,2755584.0,Aquí Entre Nos,6.0,2.755584e+06,NaN,NaN
4,113.0,"[{'name': 'Columbia Pictures Corporation', 'id...",1986-06-18,115103979.0,"The Karate Kid, Part II",5.9,1.018619e+06,NaN,NaN


In [10]:
# Initialising db and inserting dataframe
from sqlalchemy import create_engine

# Settings as shown in docker-compose.yml
engine = create_engine('postgresql://postgres:c@localhost:5432/my_database')
con = engine.connect()

In [11]:
table_name = 'TrueLayerDataEngineerChallenge'
full_df.to_sql(table_name, con, index=False)

In [12]:
print(engine.table_names())

['TrueLayerDataEngineerChallenge']
